## FYP Notebook

**Libraries**

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import pickle

In [ ]:
predictions = pd.read_csv("als-1.csv")

In [ ]:
predictions

,userId,movieId,rating,prediction
0,673,148,5.0,2.943212
1,4387,148,1.0,2.531729
2,2383,148,2.0,2.703885
3,1242,148,3.0,2.788594
4,840,148,1.0,3.246411
...,...,...,...,...
249660,2280,3910,3.0,4.124915
249661,1753,3910,4.0,3.976731
249662,1449,3910,4.0,3.354278
249663,3371,3910,1.0,3.737503


In [ ]:
rmse = mean_squared_error(predictions['rating'], predictions['prediction'], squared = False)

Original RMSE Error using ALS Algorithm

In [ ]:
rmse

0.8821783728999645

Importing extracted CNN Features

In [ ]:
features_path = '/content/drive/MyDrive/FYP/reduced_features_vgg16.pickle'

In [ ]:
with open(features_path, 'rb') as file:
    features = pickle.load(file)

In [ ]:
features

{'3203.jpg': [-631.20751953125,
  2171.90087890625,
  -485.4083557128906,
  587.7118530273438,
  418.2117919921875,
  -1517.63525390625,
  -344.65802001953125,
  -884.6752319335938,
  -357.16265869140625,
  -657.9949951171875,
  76.93904876708984,
  -332.353515625,
  -108.08309936523438,
  185.519287109375,
  599.1200561523438,
  -834.6455078125,
  347.7322082519531,
  -367.2737121582031,
  -180.21786499023438,
  346.176025390625,
  1184.8480224609375,
  131.02964782714844,
  465.50653076171875,
  620.6790771484375,
  -154.50070190429688,
  192.426025390625,
  109.59074401855469,
  857.2487182617188,
  -217.38302612304688,
  -330.2127685546875,
  249.4781951904297,
  -105.95547485351562],
 '3324.jpg': [3718.248046875,
  383.656494140625,
  916.211181640625,
  1020.4619140625,
  -641.587646484375,
  -338.4463195800781,
  1518.05322265625,
  423.389404296875,
  722.6340942382812,
  221.9889678955078,
  792.2378540039062,
  866.7877197265625,
  -263.05999755859375,
  -1225.5618896484375,


In [ ]:
type(features)

dict

In [ ]:
poster_features_df = pd.DataFrame(list(features.items()), columns=['movieId', 'poster_feature'])

In [ ]:
poster_features_df

,movieId,poster_feature
0,3203.jpg,"[-631.20751953125, 2171.90087890625, -485.4083..."
1,3324.jpg,"[3718.248046875, 383.656494140625, 916.2111816..."
2,3780.jpg,"[182.28121948242188, -2418.23486328125, 1008.1..."
3,3738.jpg,"[-1270.514892578125, -568.142822265625, -141.8..."
4,3608.jpg,"[439.02423095703125, -895.1201171875, -80.4989..."
...,...,...
3811,2036.jpg,"[1364.3526611328125, -608.7848510742188, 777.3..."
3812,1902.jpg,"[930.9751586914062, 2196.090576171875, -1277.7..."
3813,1795.jpg,"[1343.83642578125, -14.392576217651367, -654.2..."
3814,2051.jpg,"[2104.60546875, -256.8797302246094, 1473.47534..."


In [ ]:
poster_features_df['movieId'] = poster_features_df['movieId'].str.replace('.jpg', '')

<ipython-input-110-a2d707a2d164>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  poster_features_df['movieId'] = poster_features_df['movieId'].str.replace('.jpg', '')


In [ ]:
poster_features_df

,movieId,poster_feature
0,3203,"[-631.20751953125, 2171.90087890625, -485.4083..."
1,3324,"[3718.248046875, 383.656494140625, 916.2111816..."
2,3780,"[182.28121948242188, -2418.23486328125, 1008.1..."
3,3738,"[-1270.514892578125, -568.142822265625, -141.8..."
4,3608,"[439.02423095703125, -895.1201171875, -80.4989..."
...,...,...
3811,2036,"[1364.3526611328125, -608.7848510742188, 777.3..."
3812,1902,"[930.9751586914062, 2196.090576171875, -1277.7..."
3813,1795,"[1343.83642578125, -14.392576217651367, -654.2..."
3814,2051,"[2104.60546875, -256.8797302246094, 1473.47534..."


Normalizing values to make computation easy

In [ ]:
def normalize_array(arr):
    min_val = np.min(arr)
    max_val = np.max(arr)

    # Avoid division by zero by checking if the range is non-zero
    if max_val != min_val:
        normalized_arr = (arr - min_val) / (max_val - min_val)
    else:
        # If the range is zero, set all values to 0.5 (midpoint)
        normalized_arr = np.full_like(arr, 0.5)

    return normalized_arr

In [ ]:
def normalize_feature_vectors(feature_vectors):
    return [normalize_array(vector) for vector in feature_vectors]

In [ ]:
poster_features_df['normalized_feature_vector'] = normalize_feature_vectors(poster_features_df['poster_feature'])

In [ ]:
print(poster_features_df[['movieId', 'normalized_feature_vector']])

     movieId                          normalized_feature_vector
0       3203  [0.24025452047796703, 1.0, 0.2797714566374235,...
1       3324  [1.0, 0.32550166864236074, 0.4332231818756346,...
2       3780  [0.5812668823356424, 0.0, 0.7658612986403217, ...
3       3738  [0.0, 0.3181536244155367, 0.511230179062851, 0...
4       3608  [0.5753522089475229, 0.1029852578934018, 0.391...
...      ...                                                ...
3811    2036  [1.0, 0.19373516929327175, 0.7601338631834944,...
3812    1902  [0.6460172090704748, 0.8851908759848426, 0.228...
3813    1795  [1.0, 0.369672711840508, 0.07272954234926533, ...
3814    2051  [1.0, 0.25704959044625636, 0.8014392016917434,...
3815    2371  [0.8852912423352572, 0.8183696440290664, 0.722...

[3816 rows x 2 columns]


In [ ]:
poster_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816 entries, 0 to 3815
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   movieId                    3816 non-null   object
 1   poster_feature             3816 non-null   object
 2   normalized_feature_vector  3816 non-null   object
dtypes: object(3)
memory usage: 89.6+ KB


In [ ]:
poster_features_df['movieId'] = pd.to_numeric(poster_features_df['movieId'], errors='coerce')

In [ ]:
merged_df = pd.merge(predictions, poster_features_df, on='movieId', how='inner')

In [ ]:
merged_df

,userId,movieId,rating,prediction,poster_feature,normalized_feature_vector
0,673,148,5.0,2.943212,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395..."
1,4387,148,1.0,2.531729,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395..."
2,2383,148,2.0,2.703885,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395..."
3,1242,148,3.0,2.788594,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395..."
4,840,148,1.0,3.246411,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395..."
...,...,...,...,...,...,...
249075,2280,3910,3.0,4.124915,"[552.631103515625, 236.0892791748047, -295.777...","[0.8513405593132893, 0.6969868758834742, 0.437..."
249076,1753,3910,4.0,3.976731,"[552.631103515625, 236.0892791748047, -295.777...","[0.8513405593132893, 0.6969868758834742, 0.437..."
249077,1449,3910,4.0,3.354278,"[552.631103515625, 236.0892791748047, -295.777...","[0.8513405593132893, 0.6969868758834742, 0.437..."
249078,3371,3910,1.0,3.737503,"[552.631103515625, 236.0892791748047, -295.777...","[0.8513405593132893, 0.6969868758834742, 0.437..."


In [ ]:
poster_feature_columns = ['feature_' + str(i) for i in range(len(merged_df['normalized_feature_vector'][0]))]
merged_df[poster_feature_columns] = pd.DataFrame(merged_df['normalized_feature_vector'].tolist(), index=merged_df.index)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
merged_df

,userId,movieId,rating,prediction,poster_feature,normalized_feature_vector,feature_0,feature_1,feature_2,feature_3,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,673,148,5.0,2.943212,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395...",0.209787,1.000000,0.195656,0.137711,...,0.448945,0.387475,0.391159,0.396294,0.193572,0.175393,0.258615,0.349546,0.454274,0.175047
1,4387,148,1.0,2.531729,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395...",0.209787,1.000000,0.195656,0.137711,...,0.448945,0.387475,0.391159,0.396294,0.193572,0.175393,0.258615,0.349546,0.454274,0.175047
2,2383,148,2.0,2.703885,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395...",0.209787,1.000000,0.195656,0.137711,...,0.448945,0.387475,0.391159,0.396294,0.193572,0.175393,0.258615,0.349546,0.454274,0.175047
3,1242,148,3.0,2.788594,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395...",0.209787,1.000000,0.195656,0.137711,...,0.448945,0.387475,0.391159,0.396294,0.193572,0.175393,0.258615,0.349546,0.454274,0.175047
4,840,148,1.0,3.246411,"[-399.056640625, 2595.2607421875, -452.6015319...","[0.20978693018089747, 1.0, 0.19565620602468395...",0.209787,1.000000,0.195656,0.137711,...,0.448945,0.387475,0.391159,0.396294,0.193572,0.175393,0.258615,0.349546,0.454274,0.175047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249075,2280,3910,3.0,4.124915,"[552.631103515625, 236.0892791748047, -295.777...","[0.8513405593132893, 0.6969868758834742, 0.437...",0.851341,0.696987,0.437636,0.609575,...,0.467200,0.596890,0.498390,0.464768,0.754326,0.772314,0.459607,0.618118,0.383681,0.521351
249076,1753,3910,4.0,3.976731,"[552.631103515625, 236.0892791748047, -295.777...","[0.8513405593132893, 0.6969868758834742, 0.437...",0.851341,0.696987,0.437636,0.609575,...,0.467200,0.596890,0.498390,0.464768,0.754326,0.772314,0.459607,0.618118,0.383681,0.521351
249077,1449,3910,4.0,3.354278,"[552.631103515625, 236.0892791748047, -295.777...","[0.8513405593132893, 0.6969868758834742, 0.437...",0.851341,0.696987,0.437636,0.609575,...,0.467200,0.596890,0.498390,0.464768,0.754326,0.772314,0.459607,0.618118,0.383681,0.521351
249078,3371,3910,1.0,3.737503,"[552.631103515625, 236.0892791748047, -295.777...","[0.8513405593132893, 0.6969868758834742, 0.437...",0.851341,0.696987,0.437636,0.609575,...,0.467200,0.596890,0.498390,0.464768,0.754326,0.772314,0.459607,0.618118,0.383681,0.521351


In [ ]:
merged_df= merged_df.drop('poster_feature', axis = 1)
merged_df= merged_df.drop('normalized_feature_vector', axis = 1)

In [ ]:
merged_df.to_csv("/content/drive/MyDrive/FYP/normalized_combined_dataset.csv")

In [ ]:
X = merged_df.drop('rating', axis = 1)
y = merged_df['rating']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
rmse_new = mean_squared_error(y_pred, y_test, squared = False)

In [ ]:
rmse_new

0.8609325002873311

In [ ]:
rmse_old = mean_squared_error(X['prediction'], y, squared = False)

In [ ]:
rmse_old

0.8821363890053364

In [ ]:
improved_predictions = pd.DataFrame({
    'userId': merged_df['userId'],
    'movieId': merged_df['movieId'],
    'Actual_rating': merged_df['rating'],
    'Prediction by ALS': merged_df['prediction'],
    'Prediction by Hybrid Model': model.predict(X),
})

In [ ]:
improved_predictions

,userId,movieId,Actual_rating,Prediction by ALS,Prediction by Hybrid Model
0,673,148,5.0,2.943212,3.068117
1,4387,148,1.0,2.531729,2.633244
2,2383,148,2.0,2.703885,2.814619
3,1242,148,3.0,2.788594,2.903668
4,840,148,1.0,3.246411,3.392209
...,...,...,...,...,...
249075,2280,3910,3.0,4.124915,4.337502
249076,1753,3910,4.0,3.976731,4.178546
249077,1449,3910,4.0,3.354278,3.513250
249078,3371,3910,1.0,3.737503,3.925037


In [ ]:
rmse_old = mean_squared_error(improved_predictions['Prediction by ALS'], improved_predictions['Actual_rating'], squared = False)

In [ ]:
rmse_old

0.8821363890053364

In [ ]:
rmse_new = mean_squared_error(improved_predictions['Prediction by Hybrid Model'], improved_predictions['Actual_rating'], squared = False)

In [ ]:
rmse_new

0.864706711034256

In [ ]:
improved_predictions.sample()

,userId,movieId,Actual_rating,Prediction by ALS,Prediction by Hybrid Model
33482,2109,2629,1.0,2.352765,2.42691
